# Unit 2: (Theory Only) Hudi Table Types & Query Types

In unit 1, we learned to read existing an Hudi dataset. 


In this module, we will learn concepts about-
1. Hudi table types 
2. Hudi query types

This unit does not have any lab component and builds a knowledge foundation for the subsequent labs.

The documentation in this lab is sourced from public Apache Hudi docs.

In [2]:
%%html
<style>
table {float:left}
</style>

# 1. Terminology Level-set

#### Compaction

In the context of Hudi, it is the process of merging small delta logs into larger Parquet snapshot files, periodically.

# 2. Hudi Table Types at a glance

Hudi table types define how data is indexed & laid out on the DFS and how the above primitives and timeline activities are implemented on top of such organization (i.e how data is written).

Hudi supports the following table types.

#### Copy On Write Table Type
Stores data using exclusively columnar file formats (e.g parquet). Updates simply version & rewrite the files by performing a synchronous merge during write. This is the default table type in Hudi, if not specified explicitly.


#### Merge On Read Table Type 
Stores data using a combination of columnar (e.g parquet) + row based (e.g avro) file formats. Updates are logged to delta files & later compacted to produce new versions of columnar files synchronously or asynchronously.


# 3. Architectural considerations when choosing Hudi Table Types

There are tradeoffs to table types. The following are architectural considerations-

| Trade off | Copy on Write | Merge on Read |
| :- | :- | :- |
| Data Latency | Higher | Lower |
| Query Latency | Lower | Higher |
| Update cost (I/O) | Higher (rewrite entire parquet) | Lower (append to delta log) |
| Parquet File Size | Smaller (high update(I/0) cost) | Larger (low update cost) |
| Write Amplification* | Higher | Lower (depending on compaction strategy) |


*Write amplification (WA) is an undesirable phenomenon where the actual amount of information physically written to the storage media is a multiple of the logical amount intended to be written.

# 4. Deeper Dive into Copy on Write Table Type


File slices in Copy-On-Write table only contain the base/columnar file and each commit produces new versions of base files. In other words, we implicitly compact on every commit, such that only columnar data exists. As a result, the write amplification (number of bytes written for 1 byte of incoming data) is much higher, where read amplification is zero. This is a much desired property for analytical workloads, which is predominantly read-heavy.

Following illustrates how this works conceptually, when data written into copy-on-write table and two queries running on top of it.
![alternatvie text](https://hudi.apache.org/assets/images/hudi_cow-9750b5f006646e2d1874ad18b355d200.png)

As data gets written, updates to existing file groups produce a new slice for that file group stamped with the commit instant time, while inserts allocate a new file group and write its first slice for that file group. 

SQL queries running against such a table (eg: select count(*) counting the total records in that partition), first checks the timeline for the latest commit and filters all but latest file slices of each file group. As you can see, an old query does not see the current inflight commit's files color coded in pink, but a new query starting after the commit picks up the new data. Thus queries are immune to any write failures/partial writes and only run on committed data.

The intention of copy on write table, is to fundamentally improve how tables are managed today through:
- First class support for atomically updating data at file-level, instead of rewriting whole tables/partitions
- Ability to incremental consume changes, as opposed to wasteful scans or fumbling with heuristics
- Tight control of file sizes to keep query performance excellent (small files hurt query performance considerably).

# 5. Deeper Dive into Merge On Read Table

Merge on read table is a superset of copy on write, in the sense it still supports read optimized queries of the table by exposing only the base/columnar files in latest file slices. Additionally, it stores incoming upserts for each file group, onto a row based delta log, to support snapshot queries by applying the delta log, onto the latest version of each file id on-the-fly during query time. Thus, this table type attempts to balance read and write amplification intelligently, to provide near real-time data. 

The most significant change here, would be to the compactor, which now carefully chooses which delta log files need to be compacted onto their columnar base file, to keep the query performance in check (larger delta log files would incur longer merge times with merge data on query side)

Following illustrates how the table works, and shows two types of queries - snapshot query and read optimized query.

![alternatvie text](https://hudi.apache.org/assets/images/hudi_mor-5f9da4e0c57c9ee20b74b31c035ba0e6.png)


There are lot of interesting things happening in this example, which bring out the subtleties in the approach.

- We now have commits every 1 minute or so, something we could not do in the other table type.
- Within each file id group, now there is an delta log file, which holds incoming updates to records in the base columnar files. In the example, the delta log files hold all the data from 10:05 to 10:10. The base columnar files are still versioned with the commit, as before. Thus, if one were to simply look at base files alone, then the table layout looks exactly like a copy on write table.
- A periodic compaction process reconciles these changes from the delta log and produces a new version of base file, just like what happened at 10:05 in the example.
- There are two ways of querying the same underlying table: Read Optimized query and Snapshot query, depending on whether we chose query performance or freshness of data.
- The semantics around when data from a commit is available to a query changes in a subtle way for a read optimized query. Note, that such a query running at 10:10, wont see data after 10:05 above, while a snapshot query always sees the freshest data.
- When we trigger compaction & what it decides to compact hold all the key to solving these hard problems. By implementing a compacting strategy, where we aggressively compact the latest partitions compared to older partitions, we could ensure the read optimized queries see data published within X minutes in a consistent fashion.
- The intention of merge on read table is to enable near real-time processing directly on top of DFS, as opposed to copying data out to specialized systems, which may not be able to handle the data volume. There are also a few secondary side benefits to this table such as reduced write amplification by avoiding synchronous merge of data, i.e, the amount of data written per 1 bytes of data in a batch



# 6. Hudi Query Types at a glance

Hudi supports the following query types-

#### Snapshot Queries 
Queries see the latest snapshot of the table as of a given commit or compaction action. In case of merge on read table, it exposes near-real time data(few mins) by merging the base and delta files of the latest file slice on-the-fly. For copy on write table, it provides a drop-in replacement for existing parquet tables, while providing upsert/delete and other write side features.


#### Incremental Queries 
Queries only see new data written to the table, since a given commit/compaction. This effectively provides change streams to enable incremental data pipelines.


#### Read Optimized Queries
Queries see the latest snapshot of table as of a given commit/compaction action. Exposes only the base/columnar files in latest file slices and guarantees the same columnar query performance compared to a non-hudi columnar table.
Following table summarizes the trade-offs between the different query types.


# 7. Architectural Considerations for choosing Hudi Query Types

| Trade off | Snapshot queries | Read-optimized queries |
| :- | :- | :- |
| Data Latency | Lower | Higher |
| Query Latency | Higher (merge base / columnar file + row based delta / log files)	| Lower (raw base / columnar file performance)|

This concludes the unit 2. Proceed to the next notebook.

In [3]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/lib/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
/usr/lib/spark/python/pyspark/context.py:561: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  warnings.warn(
